In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
!git clone https://github.com/n1teshy/poet2 > /dev/null
!mv poet2/* . && rm -rf poet2 > /dev/null
# !ls drive/MyDrive/checkpoints/en-hi
!mv drive/MyDrive/datasets/poet2* .
!ls

In [ ]:
import torch
from core.datasets.text import GenerativeDataset
from core.tokenizers.regex import RegexTokenizer
from core.models import Generator
from core.utils import get_param_count

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 16
MODEL_D = 128
MAX_LEN = 512
CONTEXT_SIZE = 512
LAYERS = 4
HEADS = 4

In [ ]:
tokenizer = RegexTokenizer()
tokenizer.load("tokenizer/poet2_tokenizer.model")
dataset = GenerativeDataset("poet2_71683.pkl", BATCH_SIZE, device, tokenizer=tokenizer, from_pickle=True)
model = Generator(tokenizer.size, MODEL_D, MAX_LEN, LAYERS, HEADS, tokenizer.pad_id, device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)
print(get_param_count(model) / 1e6)

In [ ]:
idx = 0
while True:
    batch = dataset.next_batch()
    if batch is None:
        break
    _, loss = model(batch[0], batch[1])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if idx % 5 == 0:
        print(loss.item())
    idx += 1